<a href="https://colab.research.google.com/github/2403a510c9-hub/DAV/blob/main/week_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =====================================================
# 📈 E-commerce Sales Dashboard with Plotly
# =====================================================

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -----------------------------------------
# 1️⃣ Create Sample Data (as provided)
# -----------------------------------------
data = pd.DataFrame({
    'category': ['Electronics', 'Clothing'] * 50,
    'region': ['North', 'South', 'East', 'West'] * 25,
    'sales_volume': [100, 200, 150, 300] * 25,
    'revenue': [5000, 8000, 6000, 12000] * 25,
    'customer_rating': [4.5, 3.8, 4.2, 4.0] * 25,
    'month': ['Jan', 'Feb', 'Mar', 'Apr'] * 25
})

# -----------------------------------------
# 2️⃣ Create a figure with two subplots
# -----------------------------------------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Sales Volume vs Revenue (by Region)",
                    "Seasonal Trends by Category"),
    specs=[[{"type": "scatter"}, {"type": "heatmap"}]]
)

# -----------------------------------------
# 3️⃣ Scatter Plot: Sales Volume vs Revenue
# -----------------------------------------
for region in data['region'].unique():
    region_data = data[data['region'] == region]
    fig.add_trace(
        go.Scatter(
            x=region_data['sales_volume'],
            y=region_data['revenue'],
            mode='markers',
            name=region,
            marker=dict(size=10, opacity=0.7),
            # 🟡 Hover tooltip showing customer rating
            hovertemplate=(
                f"<b>Region:</b> {region}<br>" +
                "Sales Volume: %{x}<br>" +
                "Revenue: %{y}<br>" +
                "Customer Rating: %{customdata}<extra></extra>"
            ),
            customdata=region_data['customer_rating']
        ),
        row=1, col=1
    )

# -----------------------------------------
# 4️⃣ Heatmap: Seasonal Trends by Category
# -----------------------------------------
# Compute average revenue per category per month
heatmap_data = (
    data.groupby(['category', 'month'])['revenue']
    .mean()
    .reset_index()
    .pivot(index='category', columns='month', values='revenue')
)

# Add heatmap
fig.add_trace(
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='YlGnBu',
        hovertemplate=(
            "<b>Category:</b> %{y}<br>" +
            "Month: %{x}<br>" +
            "Avg Revenue: %{z}<extra></extra>"
        )
    ),
    row=1, col=2
)

# -----------------------------------------
# 5️⃣ Add Dropdown Menus for Month Filtering
# -----------------------------------------
months = sorted(data['month'].unique())

dropdown_buttons = [
    {
        'label': month,
        'method': 'update',
        'args': [
            {
                'visible': [
                    region == region_name and (data['month'] == month).any()
                    for region_name in data['region'].unique()
                    for region in data['region'].unique()
                ] + [True]  # Keep heatmap visible
            },
            {'title': f"Dashboard - {month}"}
        ]
    }
    for month in months
]

fig.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.4,
        'xanchor': 'left',
        'y': 1.2,
        'yanchor': 'top'
    }],
    height=600,
    width=1100,
    title_text="📊 E-commerce Sales Dashboard",
    template="plotly_white"
)

# -----------------------------------------
# 6️⃣ Final Display
# -----------------------------------------
fig.show()
